# Multi Fabric Commissioning

_This notebook walks through a flow that sets up two separate commissioners on two different fabrics and commissions a target device onto those fabrics._

### Imports

Let's first begin by setting up by importing some key modules that are needed to make it easier for us to interact with the Matter stack.

In [1]:
from rich import print
from rich.pretty import pprint
from rich import inspect
import logging
import chip
import chip.logging
from chip import ChipDeviceCtrl
import chip.clusters as Clusters
from chip.ChipReplStartup import *
from chip.ChipStack import *
import subprocess, sys

### Initialization

Next, let's initialize the REPL environment and the Matter Stack.

In [2]:
ReplInit()
chipStack = ChipStack()

──────────────────────────────────────── Matter REPL ────────────────────────────────────────

    
            Welcome to the Matter Python REPL!
    
            For help, please type matterhelp()
    
            To get more information on a particular object/class, you can pass
            that into matterhelp() as well.
    
            

─────────────────────────────────────────────────────────────────────────────────────────────

2021-12-28 23:29:21 johnsj-macbookpro1.roam.corp.google.com root[84626] CRITICAL Loading configuration from /tmp/repl-storage.json...
2021-12-28 23:29:21 johnsj-macbookpro1.roam.corp.google.com chip.DL[84626] ERROR MAC is not known, using a default.


### Launch Server

Let's launch an instance of the `chip-all-clusters-app`.

In [3]:
subprocess.Popen(['killall', 'chip-all-clusters-app'])

<subprocess.Popen object at 0x1110fca00>

No matching processes belonging to you were found


In [5]:
process = subprocess.Popen('./out/debug/chip-all-clusters-app', stdout=subprocess.DEVNULL)

FileNotFoundError: [Errno 2] No such file or directory: './out/debug/chip-all-clusters-app'

### Create Controller on Fabric A

In [16]:
devCtrl = ChipDeviceCtrl.ChipDeviceController(controllerNodeId=1)

### Commission Target

In [17]:
devCtrl.CommissionIP(b'127.0.0.1', 20202021, 1)

2021-12-28 16:30:57 johnsj-macbookpro1.roam.corp.google.com chip.CTL[96043] ERROR Unable to find country code, defaulting to WW


Established CASE with Device


True

### Read out the Fabric List to validate

In [18]:
await devCtrl.ReadAttribute(1, [(Clusters.OperationalCredentials)])

Node address has been updated
Commissioning complete


{
│   0: {
│   │   <class 'chip.clusters.Objects.OperationalCredentials'>: {
│   │   │   <class 'chip.clusters.Objects.OperationalCredentials.Attributes.FabricsList'>: [
│   │   │   │   FabricDescriptor(
│   │   │   │   │   fabricIndex=1,
│   │   │   │   │   rootPublicKey=b'\x040\xa5d\x8eX4]\xa1\xe18\x04\x9b\xf2E\x8eh\x13dwQ\xd9\x19\xa5\xe4\x88h\xe5q\xacPCtY\xb9\xcc\x94KQ\xa85\xfa(\x07\x80q+\xa6\x9a%\x8d\x1f\xf4MJ\xa0\x9e[\xafW0"\x82\xf4\xbb',
│   │   │   │   │   vendorId=39840,
│   │   │   │   │   fabricId=3,
│   │   │   │   │   nodeId=1,
│   │   │   │   │   label=''
│   │   │   │   )
│   │   │   ],
│   │   │   <class 'chip.clusters.Objects.OperationalCredentials.Attributes.SupportedFabrics'>: 16,
│   │   │   <class 'chip.clusters.Objects.OperationalCredentials.Attributes.CommissionedFabrics'>: 1,
│   │   │   <class 'chip.clusters.Objects.OperationalCredentials.Attributes.TrustedRootCertificates'>: [
│   │   │   │   b'\x150\x01\x01\x00$\x02\x017\x03$\x14\x00$\x15\x03\x18&\x04\x80"\x81\'&\x05\x80%M:7\x06$\x14\x00$\x15\x03\x18$\x07\x01$\x08\x010\tA\x040\xa5d\x8eX4]\xa1\xe18\x04\x9b\xf2E\x8eh\x13dwQ\xd9\x19\xa5\xe4\x88h\xe5q\xacPCtY\xb9\xcc\x94KQ\xa85\xfa(\x07\x80q+\xa6\x9a%\x8d\x1f\xf4MJ\xa0\x9e[\xafW0"\x82\xf4\xbb7\n5\x01)\x01\x18$\x02`0\x04\x14j\xf4P\xfa\x1dE\xff28AZ\xc9\x05]u=\x07X\x91\\0\x05\x14j\xf4P\xfa\x1dE\xff28AZ\xc9\x05]u=\x07X\x91\\\x180\x0b@\xbc@\xe6\xa9\xc1\xecjP!\xb1N\xc6_\xe1k\xaf\xc2\xc1~\xb5\xa38\xb3\x19\t\xbd\xbd\x96\xfcV\xb6\x07\xfe\xec\xc8q"\x9b\xadX\xdfg\xfb\xbaW\x90(\xfe\xeb\x14oqeV\x119\x06\xae)Q\x0e+G\x89\x18'
│   │   │   ],
│   │   │   <class 'chip.clusters.Objects.OperationalCredentials.Attributes.CurrentFabricIndex'>: 1,
│   │   │   <class 'chip.clusters.Objects.OperationalCredentials.Attributes.ClusterRevision'>: 1
│   │   }
│   }
}

Note that the `FabricsList` contains just a single item with a `fabriIndex` of 1, and a `fabricId` of 1.

### Create Controller on Fabric B

In [ ]:
devCtrl2 = ChipDeviceCtrl.ChipDeviceController(controllerNodeId=30)

### Open Commissioning Window

The target right now doesn't accept commissioning requests. So let's go ahead and open the commissioning window to permit the second controller on Fabric B to commission the target. This request has to originate from the 1st controller.

In [ ]:
await devCtrl.SendCommand(1, 0, Clusters.AdministratorCommissioning.Commands.OpenBasicCommissioningWindow(100))

### Commission Target on Fabric B

In [ ]:
devCtrl2.CommissionIP(b'127.0.0.1', 20202021, 1)

### Read out the Fabric List to validate

In [ ]:
await devCtrl2.ReadAttribute(1, [(Clusters.OperationalCredentials)])

Note that the FabricsList contains two items now!